In [ ]:
from flask import Flask, render_template, redirect, request, url_for, session, flash
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField, BooleanField, TextAreaField
from wtforms.validators import InputRequired, Email, Length, ValidationError
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
from datetime import datetime as dt

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///web.db'

db = SQLAlchemy(app)
bcrypt = Bcrypt(app)
app.config['SECRET_KEY'] = 'thisisasecretkey'

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_message = "You need to login again."
login_manager.refresh_view = 'login'
login_manager.login_view = 'login'

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

    
class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(15), unique=True)
    email = db.Column(db.String(50), unique=True)
    password = db.Column(db.String(80))
    
    def __repr__(self):
        return '<User %r>' %self.id


class Task(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    content = db.Column(db.String(200), nullable=False)
    date_created = db.Column(db.DateTime, default=dt.utcnow)

    def __repr__(self):
        return '<Task %r>' % self.id
    

db.create_all()


class JoinForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired()], render_kw={"placeholder": "Username"})
    email = StringField("Email", validators=[InputRequired()], render_kw={"placeholder": "example@gmail.com"})
    password = PasswordField("Password", validators=[InputRequired()], render_kw={"placeholder": "********"})
    submit = SubmitField("Sign Up")

    def validate_username(self, username):
        existing_user_username = User.query.filter_by(username=username.data).first()
        if existing_user_username:
            raise ValidationError("That username already exists. Please choose a different one.")

    def validate_email(self, email):
        existing_user_email = User.query.filter_by(email=email.data).first()
        if existing_user_email:
            raise ValidationError("That email address already exists. Please choose a different one.")

class LoginForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired()], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired()], render_kw={"placeholder":  "Password"})
    submit = SubmitField("Login")


################################################################################################################


@app.route("/")
def index():
    return render_template('index.html')

@app.route("/comm")
def comm():
    return render_template('comm.html')

@app.route("/post")
def post():
    return render_template('post.html')

@app.route("/guide")
def guide():
    return render_template('guide.html')

@app.route("/shop")
def shop():
    return render_template('shop.html')

################################################################################################################

@app.route('/task', methods=['POST', 'GET'])
@login_required
def task():
    if request.method =='POST':
        added_task = request.form['content']
        new_task = Task(content=added_task)
        
        try:
            db.session.add(new_task)
            db.session.commit()
            print("Added!")
            return redirect("/task")
        except Exception as e:
            print("Exception occurred.",e)
            return 'Fail to add task.'
    else:
        tasks = Task.query.order_by(Task.date_created).all()
        return render_template('task.html', tasks=tasks)
    

@app.route('/delete/<int:id>')
@login_required
def delete(id):
    task_to_delete = Task.query.get_or_404(id)

    try:
        db.session.delete(task_to_delete)
        db.session.commit()
        return redirect("/task")
    except:
        return 'Fail to delete task.'

    
@app.route('/update/<int:id>', methods=['GET', 'POST'])
@login_required
def update(id):
    task = Task.query.get_or_404(id)

    if request.method == 'POST':
        task.content = request.form['content']

        try:
            db.session.commit()
            return redirect('/task')
        except:
            return 'Fail to update task.'

    else:
        return render_template('update.html', task=task)

################################################################################################################

@app.route('/login', methods=['GET','POST'])
def login():
    if request.method == "POST":
        user = User.query.filter_by(username=request.form['username']).first()
        if user:
            if bcrypt.check_password_hash(user.password, request.form['password']):
                login_user(user)
                return redirect("/task")
        flash("User does not exist, or invalid username or password.")
    return render_template('login.html')        


@app.route("/join", methods=["GET", "POST"])
def signup():
    if request.method == "POST":
        username = request.form['username']
        email = request.form['email']
        hashed_password = bcrypt.generate_password_hash(request.form['password'])
        new_user = User(username=username, email=email, password=hashed_password)     
        try:
            db.session.add(new_user)
            db.session.commit()
            print("Registered!")
            return redirect("/login")
        except Exception as e:
            print("Exception occurred.",e)
            return 'Fail to add user.'
    else:
        return render_template('join.html')

@app.route('/logout', methods=["GET","POST"])
def logout():
    session.clear()
    logout_user()
    return redirect("/login")

################################################################################################################

@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404

if __name__ == '__main__':
    app.run('localhost', 2424)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/sammy/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:2424/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Nov/2021 14:27:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2021 14:27:18] "GET /comm HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2021 14:27:19] "GET /join HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2021 14:27:19] "GET /static/assets/img/join.jpg HTTP/1.1" 200 -
